In [165]:
import napari
%gui qt5

In [166]:
import brainlit
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.viz.swc import *
import numpy as np
from skimage import io

## Loading entire neuron from AWS 


In [167]:
s3_path = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
seg_id = 2
mip = 1
df = read_s3(s3_path, seg_id, mip)
df.head()

Downloading: 100%|██████████| 1/1 [00:00<00:00, 15.73it/s]


,sample,structure,x,y,z,r,parent
0,1,0,4713.0,4470.0,3857.0,1.0,-1
1,4,192,4721.0,4445.0,3849.0,1.0,1
2,7,64,4723.0,4446.0,3851.0,1.0,4
3,8,0,4728.0,4449.0,3852.0,1.0,7
4,14,0,4746.0,4445.0,3858.0,1.0,8


In [170]:
G = df_to_graph(df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
print(G.nodes[1]['x'],G.nodes[1]['y'],G.nodes[1]['z'])

paths = graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")

Number of nodes: 1650
Number of edges: 1649


Sample 1 coordinates (x,y,z)
4713 4470 3857
The graph was decomposed into 179 paths


In [171]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(data=np.concatenate(paths, axis=0)[1024:], edge_width=2, edge_color='white', name='all_points')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', edge_width=3, name='skeleton')

<Shapes layer 'skeleton' at 0x1b2ce0690>

In [172]:
import networkx as nx

def get_bfs_subgraph(df, v_id, d_l):

    G = df_to_graph(df)
    G_undir = G.to_undirected()

    node_id = int(df.iloc[v_id]['sample']) #convert from v_id to node_id

    # Get full list of neighbors from undirected graph
    pred = nx.bfs_predecessors(G_undir,node_id,d_l)
    p = np.concatenate([np.hstack([p[0],p[1]]) for p in pred])
    succ = nx.bfs_successors(G_undir, node_id, d_l)
    s = np.concatenate([np.hstack([s[0],s[1]]) for s in succ])
    nodes = np.unique(np.hstack((p,s))).tolist()

# Return subgraph from list of neighbors
    G_sub = nx.subgraph(G, nodes)
    
    return G_sub


In [173]:
v_id = 400
d_l = 25
G_sub = get_bfs_subgraph(df,v_id,d_l)
paths_sub = graph_to_paths(G=G_sub)


In [174]:
# viewer = napari.Viewer(ndisplay=3)
x = df.iloc[v_id]['x']
y = df.iloc[v_id]['y']
z = df.iloc[v_id]['z']
v_pos = np.array([x,y,z])
viewer.add_points(data=v_pos, edge_width=20, edge_color='orange', name='vertex')
viewer.add_points(data=np.concatenate(paths_sub), edge_color='red', edge_width=2, name='subpoints')
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='red', edge_width=3, name='sub-skeleton')

<Shapes layer 'sub-skeleton' at 0x1bdbcfd10>